In [1]:
import nltk

!pip install bert-for-tf2
!pip install sentencepiece
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

     |████████████████████████████████| 40kB 1.8MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.6-cp36-none-any.whl size=30318 sha256=513f594220fb19e4af7ca42278642f8563e463331da7ea450b544690ee428135
  Stored in directory: /root/.cache/pip/wheels/07/a0/b4/75b0601ebaa41e517a797fe9cea119c789664c8408f8a74ae9
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=4f35d828aa9ffe2eddcc191c6df86f39a70839567bfe6169d43e2392e08bb2f3
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=266ec760d05fecb04804ff233a2ff453ef8963e4248442ea4e63f57d845eb17b
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 2.8MB/s 
[nltk_data] Downloading package wor

True

In [2]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.3.0
Hub version:  0.9.0


In [3]:
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)


In [4]:
MAX_SEQ_LEN=128
input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                    name="segment_ids")

In [5]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [6]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [7]:
FullTokenizer=bert.bert_tokenization.FullTokenizer

vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()

do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()

tokenizer=FullTokenizer(vocab_file,do_lower_case)

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [8]:
import pandas as pd
col_names = ['questions', 'a', 'b']
data_df = pd.read_csv("https://raw.githubusercontent.com/VIthulan/travel-text-classification/master/data/5000TravelQuestionsDataset.csv", error_bad_lines=False,header=None, names=col_names, encoding='latin-1')

In [9]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english')) 

def remove_stopwords(text):
  word_tokens = nltk.word_tokenize(text) 
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  return " ".join(filtered_sentence)

In [10]:
# Remove all the special characters
data_df['processed_questions'] = data_df['questions'].str.replace(r'\W', ' ')
    # remove all single characters
data_df['processed_questions'] = data_df['processed_questions'].str.replace(r'\s+[a-zA-Z]\s+', ' ')
    # Remove single characters from the start
data_df['processed_questions'] = data_df['questions'].str.replace(r'\^[a-zA-Z]\s+', ' ')
    # Substituting multiple spaces with single space
data_df['processed_questions'] = data_df['questions'].str.replace(r'\s+', ' ')
    # Removing prefixed 'b'
data_df['processed_questions'] = data_df['questions'].str.replace(r'^b\s+', '')
    # Remove leading, trailing spaces
data_df['processed_questions'] = data_df['questions'].str.strip()
# Stop word removal
data_df['sw_removed_questions'] = data_df.processed_questions.apply(remove_stopwords)

In [11]:
# Remove all the special characters
data_df['processed_a'] = data_df['a'].str.replace(r'\W', ' ')
    # remove all single characters
data_df['processed_a'] = data_df['a'].str.replace(r'\s+[a-zA-Z]\s+', ' ')
    # Remove single characters from the start
data_df['processed_a'] = data_df['a'].str.replace(r'\^[a-zA-Z]\s+', ' ')
    # Substituting multiple spaces with single space
data_df['processed_a'] = data_df['a'].str.replace(r'\s+', ' ')
    # Removing prefixed 'b'
data_df['processed_a'] = data_df['a'].str.replace(r'^b\s+', '')
    # Remove leading, trailing spaces
data_df['processed_a'] = data_df['a'].str.strip()

In [12]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
  lem = [lemmatizer.lemmatize(w, pos="v") for w in nltk.word_tokenize(text)]
  return " ".join(lem)

data_df["question_lemmatized_sw"] = data_df.sw_removed_questions.apply(lemmatize_text)

In [13]:
data_df.tail(10)

,questions,a,b,processed_questions,sw_removed_questions,processed_a,question_lemmatized_sw
4990,Any self sevice laundries in Costa Teguise?,TGU,TGULAU,Any self sevice laundries in Costa Teguise?,Any self sevice laundries Costa Teguise ?,TGU,Any self sevice laundries Costa Teguise ?
4991,What are the reviews on Firefly Car Rental?,TRS,TRSRNT,What are the reviews on Firefly Car Rental?,What reviews Firefly Car Rental ?,TRS,What review Firefly Car Rental ?
4992,What are the resorts in Majorca do group boat ...,ACM,ACMRES,What are the resorts in Majorca do group boat ...,What resorts Majorca group boat trips ?,ACM,What resort Majorca group boat trip ?
4993,What are the reviews on Hotel Bagues?,ACM,ACMHOT,What are the reviews on Hotel Bagues?,What reviews Hotel Bagues ?,ACM,What review Hotel Bagues ?
4994,What is a recommendation for a reasonably pric...,TRS,TRSRNT,What is a recommendation for a reasonably pric...,What recommendation reasonably priced car rent...,TRS,What recommendation reasonably price car renta...
4995,What is the best area to be based for sightsee...,TTD,TTDSIG,What is the best area to be based for sightsee...,What best area based sightseeing Palma ?,TTD,What best area base sightsee Palma ?
4996,What are the good value traditional bars and r...,FOD,FODBAR,What are the good value traditional bars and r...,What good value traditional bars restaurants B...,FOD,What good value traditional bar restaurants Ba...
4997,What are the hotels near Alicante bus station?,ACM,ACMHOT,What are the hotels near Alicante bus station?,What hotels near Alicante bus station ?,ACM,What hotels near Alicante bus station ?
4998,Where to stay in La Gomera to mountain biking?,TTD,TTDSPO,Where to stay in La Gomera to mountain biking?,Where stay La Gomera mountain biking ?,TTD,Where stay La Gomera mountain bike ?
4999,Is it possible to take a train trip from Santi...,TRS,TRSTRN,Is it possible to take a train trip from Santi...,Is possible take train trip Santiago Madrid ?,TRS,Is possible take train trip Santiago Madrid ?


In [14]:
def create_single_input(sentence,MAX_LEN):
  
  stokens = tokenizer.tokenize(sentence)
  
  stokens = stokens[:MAX_LEN]
  
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
 
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)

  return ids,masks,segments

def create_input_array(sentences):

  input_ids, input_masks, input_segments = [], [], []

  for sentence in tqdm(sentences,position=0, leave=True):
  
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)

    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [15]:
def get_model():
  x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
  x = tf.keras.layers.Dropout(0.2)(x)
  out = tf.keras.layers.Dense(7, activation="softmax", name="dense_output")(x)

  model = tf.keras.models.Model(
        inputs=[input_word_ids, input_mask, segment_ids], outputs=out)

  model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
  return model

In [16]:
create_input_array("Is Corona Spreading again?")

100%|██████████| 26/26 [00:00<00:00, 12304.18it/s]


[array([[ 101, 1045,  102, ...,    0,    0,    0],
        [ 101, 1055,  102, ...,    0,    0,    0],
        [ 101,  102,    0, ...,    0,    0,    0],
        ...,
        [ 101, 1045,  102, ...,    0,    0,    0],
        [ 101, 1050,  102, ...,    0,    0,    0],
        [ 101, 1029,  102, ...,    0,    0,    0]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)]

In [17]:
inputs=create_input_array(data_df.question_lemmatized_sw.values)
train_y = pd.get_dummies(data_df['processed_a']).values

100%|██████████| 5000/5000 [00:00<00:00, 6156.25it/s]


In [18]:
inputs

[array([[ 101, 2054, 2569, ...,    0,    0,    0],
        [ 101, 2054, 2194, ...,    0,    0,    0],
        [ 101, 2003, 3647, ...,    0,    0,    0],
        ...,
        [ 101, 2054, 9275, ...,    0,    0,    0],
        [ 101, 2073, 2994, ...,    0,    0,    0],
        [ 101, 2003, 2825, ...,    0,    0,    0]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)]

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(data_df['processed_a'])

In [20]:
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score


# cv = KFold(n_splits=10, random_state=1, shuffle=True)
# fold = 0
# accuracies = []
# for train_index, test_index in cv.split(inputs):
#     fold += 1
#     X_train, X_test = inputs[train_index], inputs[test_index]
#     y_train, y_test = train_y[train_index], train_y[test_index]
#     print("***************************************************************")
#     print("Beginning fold: ", fold)
#     model = get_model()
#     model.fit(X_train,y_train,epochs=10,batch_size=32,validation_split=0.2,shuffle=True)
#     predictions = model.predict(X_test)
#     print(predictions)
#     break
# print("Mean {:.2f} Std {:.2f}".format(np.mean(accuracies), np.std(accuracies)))





In [21]:
# model = get_model()
# model.fit(inputs,train_y,epochs=10,batch_size=32,validation_split=0.2,shuffle=True)

Epoch 1/10
125/125 [==============================] - 102s 816ms/step - loss: 2.0462 - accuracy: 0.2075 - val_loss: 1.8708 - val_accuracy: 0.1820
Epoch 2/10
125/125 [==============================] - 108s 862ms/step - loss: 1.8135 - accuracy: 0.2365 - val_loss: 1.9954 - val_accuracy: 0.0340
Epoch 3/10
125/125 [==============================] - 107s 855ms/step - loss: 1.8100 - accuracy: 0.2188 - val_loss: 1.8754 - val_accuracy: 0.1820
Epoch 4/10
125/125 [==============================] - 107s 855ms/step - loss: 1.7957 - accuracy: 0.2307 - val_loss: 1.8228 - val_accuracy: 0.3340
Epoch 5/10
125/125 [==============================] - 107s 860ms/step - loss: 1.7903 - accuracy: 0.2320 - val_loss: 1.8577 - val_accuracy: 0.0340
Epoch 6/10
125/125 [==============================] - 107s 859ms/step - loss: 1.7875 - accuracy: 0.2365 - val_loss: 1.9658 - val_accuracy: 0.0340
Epoch 7/10
125/125 [==============================] - 107s 859ms/step - loss: 1.7911 - accuracy: 0.2310 - val_loss: 1.8628 -

# Updated model

In [19]:
import tensorflow_hub as hub
import os
os.environ['TFHUB_DOWNLOAD_PROGRESS'] = "1"

BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [23]:
tokenized_questions = [tokenize_reviews(qu) for qu in data_df['questions'].values]
reviews_with_len = [[question, y[i], len(question)] for i, question in enumerate(tokenized_questions)]

import random
random.shuffle(reviews_with_len)
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

import tensorflow as tf

processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [28]:
from tensorflow.keras import layers

class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [25]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 10

DROPOUT_RATE = 0.2

NB_EPOCHS = 10

In [26]:
def create_and_compile_bert():
    text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                            embedding_dimensions=EMB_DIM,
                            cnn_filters=CNN_FILTERS,
                            dnn_units=DNN_UNITS,
                            model_output_classes=OUTPUT_CLASSES,
                            dropout_rate=DROPOUT_RATE)
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])
    
    return text_model

In [29]:
create_and_compile_bert()


In [31]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
    
cv = KFold(n_splits=10, random_state=1, shuffle=True)
fold = 0
accuracies = []
for i in range(10):
    fold += 1
    
    text_model = create_and_compile_bert()
    
    batched_dataset.shuffle(TOTAL_BATCHES)
    test_data = batched_dataset.take(TEST_BATCHES)
    train_data = batched_dataset.skip(TEST_BATCHES)
    

    text_model.fit(train_data, epochs=NB_EPOCHS)
    results = text_model.evaluate(test_data)
    print("-------")
    print("Accuracy: %s "%results[1])
    print("======================================")

Epoch 1/10
142/142 [==============================] - 10s 73ms/step - loss: 1.4198 - sparse_categorical_accuracy: 0.4712
Epoch 2/10
142/142 [==============================] - 10s 72ms/step - loss: 0.4679 - sparse_categorical_accuracy: 0.8527
Epoch 3/10
142/142 [==============================] - 10s 72ms/step - loss: 0.1411 - sparse_categorical_accuracy: 0.9595
Epoch 4/10
142/142 [==============================] - 10s 73ms/step - loss: 0.0374 - sparse_categorical_accuracy: 0.9914
Epoch 5/10
142/142 [==============================] - 10s 72ms/step - loss: 0.0200 - sparse_categorical_accuracy: 0.9956
Epoch 6/10
142/142 [==============================] - 10s 73ms/step - loss: 0.0090 - sparse_categorical_accuracy: 0.9978
Epoch 7/10
142/142 [==============================] - 10s 73ms/step - loss: 0.0051 - sparse_categorical_accuracy: 0.9989
Epoch 8/10
142/142 [==============================] - 10s 72ms/step - loss: 0.0023 - sparse_categorical_accuracy: 0.9998
Epoch 9/10
142/142 [============

In [32]:
prediction = text_model.predict(test_data)
print(prediction)


[[2.2401089e-06 3.4720811e-06 2.0220501e-05 ... 3.1581802e-08
  1.1482370e-08 8.4627096e-09]
 [7.4915340e-07 1.8612885e-05 9.9998045e-01 ... 4.9821340e-13
  1.8031030e-12 2.5423268e-12]
 [2.3011236e-05 4.0550135e-06 1.1356462e-03 ... 8.0085734e-09
  3.8873309e-09 4.2844079e-09]
 ...
 [1.4346844e-04 4.9850113e-10 1.4941151e-06 ... 2.5226081e-09
  1.0789096e-09 5.4445454e-10]
 [7.8892208e-09 1.2184405e-07 3.2098523e-08 ... 8.2881195e-13
  2.3522812e-13 2.3358862e-13]
 [9.9934047e-01 2.6382082e-05 4.2501828e-04 ... 8.4977053e-10
  3.0907494e-09 7.7335599e-10]]


In [36]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


def print_report(y_test, y_pred):
    # Classification Report
    print(classification_report(y_test,y_pred))

    acc = accuracy_score(y_test, y_pred)*100
    print('Accuracy score: %.3f' % acc)

    print('F1 Score: %.3f' % f1_score(y_test, y_pred, average='weighted'))
    
    cm = confusion_matrix(y_test, y_pred)
    print("Confustion matrix: \n{}".format(cm))